### The Hybrid Recommender System uses both visual & textual content to give the best recommendations

#### An example dataset (IMDB) movie dataset is used to highlight the capabilities of such solution
#### Such Content-based systems are suitable for the cold-start problem in product catalogues: when you have no prior data on user demand or preferences

#### 1. NLP Based Recommender engine:

In [232]:
import pandas as pd
import numpy as np
import pandas_profiling
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
import plotly.graph_objs as go
from chart_studio import plotly
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='solar')

data=pd.read_csv('IMDBdata_MainData_poster_refresh.csv')
data[['title', 'year']].head(20:40)

,title,year
0,The Shawshank Redemption,1994
1,The Godfather,1972
2,The Godfather: Part II,1974
3,The Dark Knight,2008
4,12 Angry Men,1957
5,Pulp Fiction,1994
6,The Lord of the Rings: The Return of the King,2003
7,Schindler's List,1993
8,Anand,1971
9,Drishyam,2013


In [254]:
data[['title', 'year']].loc[560:590]

,title,year
560,Dil Se..,1998
561,The Hangover,2009
562,Dear Frankie,2004
563,The Boy in the Striped Pajamas,2008
564,Glengarry Glen Ross,1992
565,X-Men: First Class,2011
566,Kung Fu Hustle,2004
567,The Little Prince,2015
568,Oceans,2009
569,Office Space,1999


In [50]:
pandas_profiling.ProfileReport(data)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [51]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/anis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to /Users/anis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package wordnet to /Users/anis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/anis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [228]:
data=data.dropna()

In [154]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(data['plot'], 20)
df2 = pd.DataFrame(common_words, columns = ['plot' , 'count'])
df2.groupby('plot').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 words in movie plot description after removing stop words')


In [155]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(data['plot'], 20)
df4 = pd.DataFrame(common_words, columns = ['plot' , 'count'])
df4.groupby('plot').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in movie plot description After removing stop words')


In [156]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(data['plot'], 20)
df6 = pd.DataFrame(common_words, columns = ['plot' , 'count'])
df6.groupby('plot').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in movie plot description after removing stop words')


In [214]:
data['word_count'] = data['plot'].apply(lambda x: len(str(x).split()))
desc_lengths = list(data['word_count'])
print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

Number of descriptions: 2221 
Average word count 26.832507879333633 
Minimum word count 7 
Maximum word count 68


In [215]:

data['word_count'].iplot(
    kind='hist',
    bins = 50,
    linecolor='black',
    xTitle='word count',
    yTitle='count',
    title='Word Count Distribution in movie plot description')

In [216]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
    
data['plot_clean'] = data['plot'].apply(clean_text)

In [217]:

data["big_text"] = data["plot_clean"].map(str) + " " + data["genre"] +" "+ data['rated'] +" "+data['production']+" "+data['country']
data.head(2)

,title,year,decade,rated,released,runtime,genre,director,writer,actors,plot,language,country,awards,poster,ratings_source,ratings_value,metascore,imdbrating,imdbvotes,imdbid,dvd,boxoffice,production,website,response,tomatourl,word_count,plot_clean,big_text
3,The Dark Knight,2008,2000,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True,http://www.rottentomatoes.com/m/the_dark_knight/,41,menace known joker emerges mysterious past wre...,menace known joker emerges mysterious past wre...
6,The Lord of the Rings: The Return of the King,2003,2000,PG-13,17 Dec 2003,201 min,"Adventure, Drama, Fantasy",Peter Jackson,"J.R.R. Tolkien (novel), Fran Walsh (screenplay...","Noel Appleby, Ali Astin, Sean Astin, David Aston",Gandalf and Aragorn lead the World of Men agai...,"English, Quenya, Old English, Sindarin","USA, New Zealand",Won 11 Oscars. Another 197 wins & 122 nominati...,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,Internet Movie Database,8.9/10,94.0,8.9,"1,304,569",tt0167260,25 May 2004,"$364,000,000",New Line Cinema,http://www.lordoftherings.net/,True,http://www.rottentomatoes.com/m/the_lord_of_th...,28,gandalf aragorn lead world men saurons army dr...,gandalf aragorn lead world men saurons army dr...


In [218]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data['big_text'])

In [219]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [220]:
# Reset index of your main DataFrame and construct reverse mapping as before
data = data.reset_index()
indices = pd.Series(data.index, index=data['title'])

In [221]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:8]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data['title'].iloc[movie_indices], [sim_scores]

In [229]:
get_recommendations('Inception', cosine_sim)

(3472                      Superman Returns
 637                                Gravity
 536                       Edge of Tomorrow
 2471    Batman v Superman: Dawn of Justice
 4529                     Jupiter Ascending
 5065          The Adventures of Pluto Nash
 268                Guardians of the Galaxy
 Name: title, dtype: object,
 [[(1495, 0.469041575982343),
   (170, 0.43656412506539943),
   (135, 0.41702882811414965),
   (1009, 0.40000000000000013),
   (1976, 0.40000000000000013),
   (2180, 0.3880570000581328),
   (54, 0.3837612894400988)]])

In [223]:
get_recommendations('Domestic Disturbance', cosine_sim)

(1572                              Dream House
 1351                                    Devil
 1159                         The Skeleton Key
 714                              Jack Reacher
 1075                            Secret Window
 1577    The Fast and the Furious: Tokyo Drift
 1239                                     Salt
 Name: title, dtype: object,
 [[(1572, 0.44038550605054416),
   (1351, 0.41403933560541256),
   (1159, 0.4050957468334665),
   (714, 0.40414518843273806),
   (1075, 0.3999999999999999),
   (1577, 0.3944053188733077),
   (1239, 0.3872983346207416)]])

In [224]:
get_recommendations('Stealth', cosine_sim)

(1648                The 6th Day
 1705         Battle Los Angeles
 1692                       2012
 54      Guardians of the Galaxy
 1495           Superman Returns
 740                 Pacific Rim
 1623                  Red Tails
 Name: title, dtype: object,
 [[(1648, 0.4181210050035455),
   (1705, 0.4160251471689219),
   (1692, 0.38516444325982174),
   (54, 0.37630890450319093),
   (1495, 0.37630890450319093),
   (740, 0.3641785203646148),
   (1623, 0.3641785203646148)]])

In [230]:
get_recommendations('Interstellar', cosine_sim)

(637                Gravity
 2791     War of the Worlds
 62            The Prestige
 466            The Martian
 1360           Eight Below
 2954    Star Trek: Nemesis
 899              True Grit
 Name: title, dtype: object,
 [[(170, 0.47633051162246687),
   (1163, 0.4629100498862757),
   (15, 0.4095141796739611),
   (108, 0.4095141796739611),
   (490, 0.40050093945740706),
   (1255, 0.40050093945740706),
   (285, 0.3818813079129867)]])